In [ ]:
#start tika server. The Tika Server is the Parser
#!java -jar "C:\Users\szubair\AppData\Local\Continuum\anaconda3\tika-server-1.22.jar"

In [26]:
# import necessary modules
import os
import tika
tika.initVM()
from tika import parser
import os.path
import pandas as pd
import numpy as np
import spacy
import nltk
from spacy.lang.en import English
nlp = English()
nlp.max_length = 10000000
from datetime import datetime
from dateparser.search import search_dates
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
import re
import time
from collections import Counter
from summa.summarizer import summarize
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"
from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
import dask.dataframe as dd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
#define a parameter for tika parsers. This declaration solves the status 422 server error
headers = {'X-Tika-PDFextractInlineImages': 'true', "X-Tika-OCRLanguage": "eng"} 

In [3]:
"""Importing all Lookup Data"""

#import project lookup data
proj_lookup = pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\proj_codes.csv", encoding = "ISO-8859-1")

#import circuit lookup table and convert to list
circuit_lookup = list(pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\circuits.csv", header=None, encoding = "ISO-8859-1")[0])

#import station lookups
code_lookup = list(pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\station-codes.csv", header = None)[0])
code_name_lookup = list(pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\station-names.csv", encoding = "ISO-8859-1")['Code'])
name_lookup = list(pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\station-names.csv", encoding = "ISO-8859-1")['Name'])


#import reservoirs lookup and convert to lists
reservoir_name = list(pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\reservoirs.csv", encoding = "ISO-8859-1", header = None)[0])
reservoir_values = list(pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\reservoirs.csv", encoding = "ISO-8859-1", header = None)[1])

#import keywords list
keywords_lookup = list(pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\keywords.csv", encoding = "ISO-8859-1", header = None)[0])

#import authors list
authors_lookup = list(pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\authors.csv", encoding = "ISO-8859-1", header = None)[0])

In [4]:
#parse text from scanned files
def ocr_pdf(file):
    images = convert_from_path(file)
    ocr_list = [pytesseract.image_to_string(x) for x in images]
    ocr = ''
    return ocr.join(ocr_list)

In [5]:
"""RETURN THE ROOT FOLDER BY EXTRACTING MID STRING"""
def find_root(path):
    return path[32: path.find("\\", 32)] #the number here is 32 because raw strings. With regular strings the number is 26

In [6]:
"""Function to extract dates from a piece of text"""
today = datetime.today()

#Match strings that contain only numbers and special characters
def special_match(strg, search=re.compile(r'[^|\&+\-%*/=!>0-9.]').search):
    return not bool(search(strg))

def find_dates(text):
    found_dates = search_dates(text, languages = ['en'], settings={'STRICT_PARSING': True})
    """Exclude Possible error matches"""
    found_dates = [x for x in found_dates if x[1] <= datetime.today() and x[1] > datetime(1940, 1, 1)] #only return dates between 1940 and today
    found_dates = [x for x in found_dates if re.fullmatch('\d{4}', x[0]) is None]  #search_dates sometimes confuses years for full dates. This excludes those errors
    found_dates = [x for x in found_dates if x[1].hour == 0] #exclude matches that includes hours, mins, secs. Most likely error match
    return found_dates[0][1]

In [7]:
#collect all the file paths in folder and return as numpy array
def return_paths(folder):
    paths = np.array([os.path.join(r, file) for r, d, f in os.walk(folder) for file in f])
    return paths

In [8]:
#Run Tika Parser on Texts
def return_parsed(paths):
    try:
        return parser.from_file(paths, headers=headers)
    except:
        return 'path error'

In [9]:
#Extract texts
def return_texts(parsed, paths):
    if 'content' in parsed and parsed['content'] is not None:
        return parsed['content'] #extract 'content' from parsed texts
    else:
        try:
            return ocr_pdf(paths) #if no 'content' from tika parser, try OCRing the document
        except:
            return "no content"   

In [10]:
#Zip Paths and Texts
def return_zip_items(paths, texts):
    return ''.join([paths, texts])

In [11]:
#Return Metadata values of document from tika parser
def return_metadata(parsed):
    try:
        return parsed['metadata']
    except:
        return "no content"    

In [12]:
#Return document filename
def return_filenames(paths):
    try:
        return os.path.basename(paths)
    except:
        return "no content"

In [13]:
#Return document datae
def return_dates(filenames, texts, metadata):
    try:
        return find_dates(filenames) #first find date in document filename
    except:
        try:
            return find_dates(texts[:1000]) #if no date in filename, search 1st 1000 characters for date
        except:
            try:
                return metadata['Last-Modified'] # if no date in 1st 1000 characters, returned date last modified
            except:
                return "no content"   

In [14]:
"""Extract Keywords from text"""
def return_keywords(zip_items):
    try:
        keywords_list = [keyword for keyword in keywords_lookup if keyword in zip_items.lower()]
        return list(dict.fromkeys(keywords_list))
    except:
        return 'no content'

In [15]:
#determine the document author
def return_authors(paths, zip_items):
    authors_list = []
    try:
        authors_list = [author.title() for author in authors_lookup if author in zip_items[:1000].lower()]
        if len(authors_list) == 0:
            authors_list.append(find_root(repr(paths)))
        return list(dict.fromkeys(authors_list))
    except:
        return 'no content'   

In [16]:
#Use gensim summarize module to summarize the document
def return_descriptions(texts):
    try:
        return summarize(texts, words=50, language='english')
    except:
        return 'no content'

In [17]:
#Search Paths and Texts for Project IDs referenced
def return_proj_ids(zip_items):
    #populate proj_codes list with all project IDs found
    proj_no = []
    proj_id_regex = r"[BDEFGTYbdefgty][ABCEFILMPRSVYZabcefilmprsvyz][-\s]?\d{4,5}|[A-Z]{3}MON[-\s]?\d{1,2}[A-Za-z]?|[A-Z]{3}WORKS[-\s]?\d{1,2}[A-Za-z]?"
    proj_no.append(re.findall(proj_id_regex , zip_items[:1000]))

    #remove duplicate occurrences
    proj_codes = [list(dict.fromkeys(x)) for x in proj_no]
        
    #reformat wrongly formatted project codes to standard
    regex1 = r"([A-Z]{3}MON)\s?(\d{1,2}[A-Za-z]?)"           #ABFMON 02(A)/ ABFMON02(A) ----> ABFMON-02(A)
    regex2 = r"([A-Z]{3}WORKS)\s?(\d{1,2}[A-Za-z]?)"         #BRGWORKS 01(A)/BRGWORKS01(A)  ---->  BRGWORKS-01(A)
    regex3 = r"([A-Z]{3}MON)-(\d{1}[A-Za-z]?)"               #ABFMON-2/ CLBMON-1A  ---->  ABFMON-02(A)
    regex4 = r"([A-Z]{3}WORKS)-(\d{1}[A-Za-z]?)"             #BRGWORKS-1/ CLBWORKS-2A  ---->  BRGWORKS-01(A)
    regex5 = r"([BDEFGTYbdefgty][ABCEFILMPRSVYZabcefilmprsvyz])\s?(\d{4,5})"   #TY7111 / TY 7111  ---->  TY-7111

    #capture rightly formatted project codes too
    regex6 = r"[A-Z]{3}MON-\d{2}[A-Za-z]?"
    regex7 = r"[A-Z]{3}WORKS-\d{2}[A-Za-z]?"
    regex8 = r"[A-Za-z]{2}-\d{4}"

    new_proj = []
    for reg in proj_codes:
        a = [re.sub(regex1, r"\1-\2", x) for x in reg]
        b = [re.sub(regex2, r"\1-\2", x) for x in reg] + a
        c = [re.sub(regex3, r"\1-0\2", x) for x in b + reg] + b
        d = [re.sub(regex4, r"\1-0\2", x) for x in c + reg] + c
        e = [re.sub(regex5, r"\1-\2", x) for x in reg] + d
        e.append(re.findall(regex6 , str(reg)))
        e.append(re.findall(regex7 , str(reg)))
        e.append(re.findall(regex8 , str(reg)))
        new_proj.append(e)
    proj_codes = new_proj
    
    #validate project codes are in project codes lookup table
    validate_codes = []
    for x in proj_codes:
        y = [str(z).upper() for z in x if str(z).upper() in list(proj_lookup['Project Code'])]
        validate_codes.append(y)
    proj_codes = []

    #remove duplicate occurrences      
    for x in validate_codes:
        proj_codes = list(dict.fromkeys(x))
    return proj_codes

In [18]:
#Use proj_lookup to find project name for all project codes
def return_proj_names(proj_ids):
    try:
        for x in proj_ids:
            proj_names = [list(proj_lookup['Project Name'])[list(proj_lookup['Project Code']).index(y)] for y in proj_ids]
        return proj_names
    except:
        return "no content"

In [19]:
#find circuits referenced in paths and texts
def return_circuits(zip_items):
    try:
        item_delimited = re.split('[|\^&+\-%*/=!>\s\-.,\\\\]', zip_items)     #delimit because some circuits are contained in other circuits e.g. 1L1 AND 1L135 
        circuit_list = [circuit for circuit in circuit_lookup if circuit in item_delimited]
        return circuit_list
    except:
        return 'no content'

In [20]:
#find station names referenced in paths and texts
def return_stations(zip_items):
    name_list = []
    name_list.append([code_name_lookup[name_lookup.index(name)] for name in name_lookup if name in zip_items.lower()])

    #remove duplicate occurrences
    for x in name_list:
        stations = list(dict.fromkeys(x))
    return stations

In [21]:
#find reservoirs referenced in paths and texts
def return_reservoirs(zip_items):
    reservoirs = []
    reservoirs = [reservoir_name[reservoir_values.index(value)] for value in reservoir_values if value in zip_items.lower()]
    return reservoirs

In [22]:
#Return prediction probability 
def prediction_probability(encoded, probas):
    return probas[encoded]

In [23]:
"""Import necessary materials for Arch Data Classification"""
#import arch training data
arch_df = pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\parsed-arch_data-training-data-2.csv", encoding = "ISO-8859-1")

#Create Vectorizer Object for Arch Classification
arch_tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

#Fit Arch Data Classifier Vectorizer
arch_y, arch_mappings =arch_df.Arch_Data.factorize()
arch_tfidf.fit_transform(arch_df.Texts).toarray()

#Import Arch Data Classifier Model
arch_clf = pickle.load(open(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\final-arch-model.sav", 'rb'))

In [24]:
"""Import necessary materials for Document Type Classification"""
#import Document Type training data
doc_df = pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\parsed-doc-type-training-data-4.csv", encoding = "ISO-8859-1")

#Create Vectorizer Object for Document Type Classification
doc_tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

#Fit Document Type Classifier Vectorizer
doc_y, doc_mappings =doc_df.Document_Type.factorize()
doc_tfidf.fit_transform(doc_df.Texts).toarray()

#mport Document Type Classifier Model
doc_clf = pickle.load(open(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\final-doc-type-model.sav", 'rb'))

In [27]:
"""Import necessary materials for Subject Area Classification"""
#import Document Type training data
sub_df = pd.read_csv(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\parsed-subject-training-data-5.csv", encoding = "ISO-8859-1")

#Create Vectorizer Object for Subject Area Classification
sub_tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

#Fit Subject Area Classifier Vectorizer
mlb = MultiLabelBinarizer()
sub_y = [set(i.split(',')) for i in sub_df.Subject_Area]
sub_y = mlb.fit_transform(sub_y)
sub_tfidf.fit_transform(sub_df.Texts).toarray()

#Import Document Type Classifier Model
sub_clf = pickle.load(open(r"\\esri-shelf\ESRIENVIROHUB\TaggerBot\subject-model-2.sav", 'rb'))

In [50]:
#Default Unclassified Subject Areas to the string "Environment"
def default_subject(Subject):
    if Subject == ():
        return 'Environment'
    else:
        return Subject

In [47]:
def tagger(inputFolder):
    
    paths = return_paths(inputFolder)
    tagger_df = pd.DataFrame({'Paths': paths})
    tagger_df = dd.from_pandas(tagger_df, npartitions=5) #Convert Pandas DataFrame to Dask DataFrame
    
    parsed = tagger_df.apply(lambda row: return_parsed(row['Paths']), axis = 1)
    tagger_df['Parsed'] = parsed 
    
    texts =  tagger_df.apply(lambda row: return_texts(row['Parsed'], row['Paths']), axis = 1)
    tagger_df['Texts'] = texts
    
    zip_items =  tagger_df.apply(lambda row: return_zip_items(row['Paths'], row['Texts']), axis = 1)
    tagger_df['Zip Items'] = zip_items
                                                          
    metadata =  tagger_df.apply(lambda row: return_metadata(row['Parsed']), axis = 1)
    tagger_df['Metadata'] = metadata
    os.system(r'taskkill /IM "java.exe" /F') #end tika process after tagging
    
    filenames =  tagger_df.apply(lambda row: return_filenames(row['Paths']), axis = 1)
    tagger_df['Filenames'] = filenames
    
    dates =  tagger_df.apply(lambda row: return_dates(row['Filenames'], row['Texts'], row['Metadata']), axis = 1)
    tagger_df['Dates'] = dates
    
    keywords =  tagger_df.apply(lambda row: return_keywords(row['Zip Items']), axis = 1)
    tagger_df['Keywords'] = keywords
    
    authors =  tagger_df.apply(lambda row: return_authors(row['Paths'], row['Zip Items']), axis = 1)
    tagger_df['Authors'] = authors
    
    #descriptions =  tagger_df.apply(lambda row: return_descriptions(row['Texts']), axis = 1)
    #tagger_df['Descriptions'] = descriptions
    
    proj_ids =  tagger_df.apply(lambda row: return_proj_ids(row['Zip Items']), axis = 1)
    tagger_df['Project IDs'] = proj_ids
    
    proj_names =  tagger_df.apply(lambda row: return_proj_names(row['Project IDs']), axis = 1)
    tagger_df['Project Names'] = proj_names
    
    circuits =  tagger_df.apply(lambda row: return_circuits(row['Zip Items']), axis = 1)
    tagger_df['Circuits'] = circuits
    
    stations =  tagger_df.apply(lambda row: return_stations(row['Zip Items']), axis = 1)
    tagger_df['Stations'] = stations
    
    reservoirs =  tagger_df.apply(lambda row: return_reservoirs(row['Zip Items']), axis = 1)
    tagger_df['Reservoirs'] = reservoirs
    tagger_df = tagger_df.compute() #Convert Dask DataFrame back to Pandas DataFrame #Exploring faster options

    arch_X = arch_tfidf.transform(tagger_df.Texts).toarray()
    tagger_df['Arch_Data'] = arch_mappings[arch_clf.predict(arch_X)]
    
    doc_X = doc_tfidf.transform(tagger_df.Texts).toarray()
    tagger_df['encoded_pred'] = doc_clf.predict(doc_X)
    tagger_df['Document_Type'] = doc_mappings[tagger_df['encoded_pred']]
    tagger_df['pred_probas'] = list(doc_clf.predict_proba(doc_X))
    tagger_df['Document_Type_Probability'] =  tagger_df.apply(lambda row: prediction_probability(row['encoded_pred'], row['pred_probas']), axis = 1)
    
    sub_X = sub_tfidf.transform(tagger_df.Texts).toarray()
    tagger_df['Subject_Area'] = mlb.inverse_transform(sub_clf.predict(sub_X))
    tagger_df['Subject_Area'] = tagger_df.apply(lambda row: default_subject(row['Subject_Area']), axis = 1)
    
    return tagger_df

In [29]:
tagger(r"\\esri-shelf\ESRIENVIROHUB\test2")

2020-02-24 09:58:07,518 [Thread-12   ] [WARNI]  Failed to see startup log message; retrying...
2020-02-24 09:58:07,528 [Thread-11   ] [WARNI]  Failed to see startup log message; retrying...
2020-02-24 09:58:07,555 [Thread-10   ] [WARNI]  Failed to see startup log message; retrying...
2020-02-24 09:58:07,556 [Thread-13   ] [WARNI]  Failed to see startup log message; retrying...
2020-02-24 09:58:12,548 [Thread-12   ] [WARNI]  Failed to see startup log message; retrying...
2020-02-24 09:58:12,554 [Thread-11   ] [WARNI]  Failed to see startup log message; retrying...
2020-02-24 09:58:12,575 [Thread-10   ] [WARNI]  Failed to see startup log message; retrying...
2020-02-24 09:58:12,592 [Thread-13   ] [WARNI]  Failed to see startup log message; retrying...


,Paths,Parsed,Texts,Zip Items,Metadata,Filenames,Dates,Keywords,Authors,Project IDs,Project Names,Circuits,Stations,Reservoirs,Arch_Data,encoded_pred,Document_Type,pred_probas,Document_Type_Probability,Subject_Area
0,\\esri-shelf\ESRIENVIROHUB\test2\1537874-004-R...,"{'status': 200, 'content': ' ...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,\\esri-shelf\ESRIENVIROHUB\test2\1537874-004-R...,"{'Author': 'cmcallister', 'Content-Type': ['ap...",1537874-004-R-Rev0-LCR indexing 2017 Report 15...,2019-01-15 00:00:00,"[agreement, als, amp, bass, bat, bathymetry, b...",[Golder Associates],[CLBMON-45],[CLBMON-45 LOW COL FISH POPULATION INDEXING],[],"[ALH, BEC, BRX, CRO, RO2, TSS, WAX]","[Arrow Lakes Reservoir, Bear Creek Reservoir]",n,0,Reports,"[0.9999998507448368, 1.1996440729850394e-08, 2...",1.000000,"(Fish and Aquatic,)"
1,\\esri-shelf\ESRIENVIROHUB\test2\CLBMON-21 Yr1...,"{'status': 200, 'content': ' ...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,\\esri-shelf\ESRIENVIROHUB\test2\CLBMON-21 Yr1...,"{'Content-Type': ['application/pdf', 'image/pn...",CLBMON-21 Yr12 2019-06-01.pdf,2019-06-01 00:00:00,"[als, amec, amec foster wheeler, amp, applicat...",[test2],[CLBMON-21],[CLBMON-21 MID COL JUVENILE STURGEON DETECTION...,[],"[LIK, REV, WHN]","[Arrow Lakes Reservoir, Arrow Lakes Reservoir,...",n,0,Reports,"[0.999998256351634, 1.4587323123447914e-07, 4....",0.999998,"(Fish and Aquatic,)"
2,\\esri-shelf\ESRIENVIROHUB\test2\CMSMON1a data...,"{'status': 200, 'content': ' ...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,\\esri-shelf\ESRIENVIROHUB\test2\CMSMON1a data...,"{'Author': 'stephanie lingard', 'Content-Type'...",CMSMON1a data report 2018 field season_Novembe...,2018-11-13T23:44:55Z,"[agreement, als, amp, bulletin, burn, char, ch...",[Instream Fisheries Research],[CMSMON-01A],[CMSMON-01A CHEAKAMUS RIVER JUVENILE OUTMIGRAN...,[],"[BCR, CMS]",[Daisy Lake Reservoir (Shadow Lake)],n,0,Reports,"[0.9994142089623389, 4.1498967900233305e-05, 0...",0.999414,"(Fish and Aquatic,)"
3,\\esri-shelf\ESRIENVIROHUB\test2\FINAL_Plan_fo...,"{'status': 200, 'content': ' ...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,\\esri-shelf\ESRIENVIROHUB\test2\FINAL_Plan_fo...,"{'Author': 'Elinor McGrath', 'Company': 'Hewle...",FINAL_Plan_for_Fish_Passage_at_Wilsey_Dam_-_Ma...,2018-05-14 20:14:08,"[acid rock drainage, advisory, agreement, als,...",[test2],[],no content,[],"[LU2, RVS, SGR, WIL]","[Arrow Lakes Reservoir, Duncan Lake Reservoir,...",n,0,Reports,"[0.9999998160530256, 6.25809278704018e-08, 5.4...",1.000000,"(Fish and Aquatic,)"
4,\\esri-shelf\ESRIENVIROHUB\test2\Anadromous Sa...,"{'status': 200, 'content': ' ...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,\\esri-shelf\ESRIENVIROHUB\test2\Anadromous Sa...,"{'Author': 'Giles Shearing', 'Content-Type': [...",Anadromous Salmon Spawning Habitat Middle Shus...,2013-04-08 00:00:00,"[agreement, als, amp, application, bat, bathym...",[test2],[],no content,[],[LU2],[],n,0,Reports,"[0.9999778668965223, 2.8360762965578354e-06, 4...",0.999978,"(Fish and Aquatic,)"


In [2]:
list_experiment = [*range(1, 1001, 200)]

In [ ]:
#concept for running TaggerBot in batches
for x in list_experiment:
    if list_experiment.index(x)+1 in list_experiment:
        print([*range(x, list_experiment[list_experiment.index(x)+1])])
    else:
        print([*range(x, 1001)])